In [256]:
import pandas as pd
import numpy as np
import random
from catboost import CatBoostRanker, Pool, MetricVisualizer, CatBoostClassifier
from copy import deepcopy

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [257]:


def featurize_df(df:pd.DataFrame) ->pd.DataFrame:
    """
    Extract more features
    """
    df["weekday"] = df["date_time"].dt.weekday
    df["week_of_year"] = df["date_time"].dt.isocalendar().week

    df["hour"] = df["date_time"].dt.hour
    df["minute"] = df["date_time"].dt.minute
    ## total time elapsed - allows model to learn continous trend over time to a degree
    df["time_epoch"] = df["date_time"].astype('int64')//1e9
    ## if we were looking at fraud: df["seconds"] = df.timestamp.dt.second
    df["early_night"] = ((df["hour"]>19) | (df["hour"]<3)) # no added value from feature
    
    df["nans_count"] = df.isna().sum(axis=1)
    
    ## we won't make any time series features for now
    ## We could add time series features per property/hotel. We'd need to check for unaries, and to add a shift/offset dependant on forecast horizon

    return df


In [258]:
HotelSearch_train=pd.read_csv("./data/HotelSearch_train_sampled.csv", index_col=0)
HotelSearch_test=pd.read_csv("./data/HotelSearch_test_sampled.csv", index_col=0)



#print(HotelSearch_orig['booking_bool'].unique())
HotelSearch_train["date_time"] = pd.to_datetime(HotelSearch_train["date_time"],infer_datetime_format=True)
#HotelSearch_train["target"] = HotelSearch_train.apply(get_target,axis=1)

HotelSearch_test["date_time"] = pd.to_datetime(HotelSearch_test["date_time"],infer_datetime_format=True)
#HotelSearch_test["target"] = HotelSearch_test.apply(get_target,axis=1)

HotelSearch_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 98361 entries, 0 to 98360
Data columns (total 56 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   srch_id                      98361 non-null  int64         
 1   date_time                    98361 non-null  datetime64[ns]
 2   site_id                      98361 non-null  int64         
 3   visitor_location_country_id  98361 non-null  int64         
 4   visitor_hist_starrating      4699 non-null   float64       
 5   visitor_hist_adr_usd         4724 non-null   float64       
 6   prop_country_id              98361 non-null  int64         
 7   prop_id                      98361 non-null  int64         
 8   prop_starrating              98361 non-null  int64         
 9   prop_review_score            98201 non-null  float64       
 10  prop_brand_bool              98361 non-null  int64         
 11  prop_location_score1         98361 non-nu

In [259]:
drop_cols=[]
drop_unary_cols = [c for c in list(HotelSearch_train)
    if HotelSearch_train[c].nunique(dropna=False) <= 1]
#print(drop_unary_cols)
target_cols = ["gross_bookings_usd","click_bool","booking_bool","Unnamed: 0"] # leaky column, and original target columns
drop_cols.extend(drop_unary_cols)
drop_cols.extend(target_cols) 
print(drop_cols)
HotelSearch_train = HotelSearch_train.drop(columns=drop_cols,errors="ignore")
HotelSearch_test  = HotelSearch_test.drop(columns=drop_cols,errors="ignore")
print(HotelSearch_train.shape)
print(HotelSearch_test.shape)


['gross_bookings_usd', 'click_bool', 'booking_bool', 'Unnamed: 0']
(98361, 53)
(147730, 53)


In [260]:
HotelSearch_train = featurize_df(HotelSearch_train)
HotelSearch_test = featurize_df(HotelSearch_test)
HotelSearch_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 147730 entries, 0 to 147729
Data columns (total 60 columns):
 #   Column                       Non-Null Count   Dtype         
---  ------                       --------------   -----         
 0   srch_id                      147730 non-null  int64         
 1   date_time                    147730 non-null  datetime64[ns]
 2   site_id                      147730 non-null  int64         
 3   visitor_location_country_id  147730 non-null  int64         
 4   visitor_hist_starrating      7770 non-null    float64       
 5   visitor_hist_adr_usd         7801 non-null    float64       
 6   prop_country_id              147730 non-null  int64         
 7   prop_id                      147730 non-null  int64         
 8   prop_starrating              147730 non-null  int64         
 9   prop_review_score            147499 non-null  float64       
 10  prop_brand_bool              147730 non-null  int64         
 11  prop_location_score1      

In [261]:
HotelSearch_train.drop(['comp3_rate',
       'comp3_inv', 'comp3_rate_percent_diff', 'comp4_inv', 'comp5_rate',
       'comp5_inv', 'comp5_rate_percent_diff', 'comp8_rate', 'comp8_inv',
       'comp8_rate_percent_diff'],axis=1).groupby(HotelSearch_train["target"]>0).mean()
HotelSearch_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 98361 entries, 0 to 98360
Data columns (total 60 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   srch_id                      98361 non-null  int64         
 1   date_time                    98361 non-null  datetime64[ns]
 2   site_id                      98361 non-null  int64         
 3   visitor_location_country_id  98361 non-null  int64         
 4   visitor_hist_starrating      4699 non-null   float64       
 5   visitor_hist_adr_usd         4724 non-null   float64       
 6   prop_country_id              98361 non-null  int64         
 7   prop_id                      98361 non-null  int64         
 8   prop_starrating              98361 non-null  int64         
 9   prop_review_score            98201 non-null  float64       
 10  prop_brand_bool              98361 non-null  int64         
 11  prop_location_score1         98361 non-nu

In [262]:
cutoff_id = HotelSearch_train["srch_id"].quantile(0.94) # 90/10 split

X_train_df=HotelSearch_train.loc[HotelSearch_train.srch_id< cutoff_id].drop(["target","date_time"],axis=1)
X_eval_df = HotelSearch_train.loc[HotelSearch_train.srch_id>= cutoff_id].drop(["target","date_time"],axis=1)
X_test_df=HotelSearch_test.drop(["target","date_time"],axis=1)

feature_cols=X_train_df.columns.tolist()
# X_train_df=HotelSearch_train.loc[HotelSearch_train.srch_id< cutoff_id]
# X_eval_df = HotelSearch_train.loc[HotelSearch_train.srch_id>= cutoff_id]

y_train_df = HotelSearch_train.loc[HotelSearch_train.srch_id< cutoff_id]["target"].astype(float)
y_eval_df = HotelSearch_train.loc[HotelSearch_train.srch_id>= cutoff_id]["target"].astype(float)
y_test_df = HotelSearch_test["target"].astype(float)

X_train = X_train_df.values
X_eval = X_eval_df.values
X_test= X_test_df.values
queries_train = X_train_df["srch_id"].values

y_train = y_train_df.values
y_eval = y_eval_df.values
y_test = y_test_df.values

queries_test = X_eval_df["srch_id"].values



max_relevance = float(np.max(y_train))
y_train /= max_relevance
y_eval /= max_relevance
y_train_df.value_counts()


0.0    89823
1.0     2628
Name: target, dtype: int64

In [263]:
print("mean relevancy train",round(y_train.mean(),4))
print("mean relevancy eval",round(y_eval.mean(),4))
print(y_eval_df.value_counts()) # check we have all 3 "labels" in subset

mean relevancy train 0.0284
mean relevancy eval 0.0293
0.0    5737
1.0     173
Name: target, dtype: int64


In [264]:
HotelSearch_train['target'].value_counts()

0    95560
1     2801
Name: target, dtype: int64

In [265]:
categorical_cols = ['prop_id',"srch_destination_id", "weekday"] # ,"week_of_year"

In [266]:
X_train_df.shape

(92451, 58)

In [267]:
#set(X_train.columns).symmetric_difference(set(HotelSearch_test.columns))
train_pool = Pool(data=X_train,
                  label = y_train,
#                  cat_features=categorical_cols,
                  group_id=queries_train
                 )

eval_pool = Pool(data=X_eval,
                  label = y_eval,
#                  cat_features=categorical_cols,
                  group_id=queries_test
                 )

In [268]:
default_parameters  = {
    'iterations': 2000,
    'custom_metric': ['NDCG', "AUC:type=Ranking",'PrecisionAt:top=10', 'RecallAt:top=10', 'MAP:top=10'], # , 'AverageGain:top=3'# 'QueryRMSE', "YetiLoss" (use with hints)
    'verbose': False,
    'random_seed': 42,
#     "task_type":"GPU",
#    "has_time":True,
    "metric_period":4,
    "save_snapshot":False,
    "use_best_model":True, # requires eval set to be set
} 

default_clf_parameters  = {
    'iterations':2000,
    'random_seed':42,
    'learning_rate':0.5,
    'custom_loss':['AUC', 'Accuracy']
}    


In [313]:
def fit_model(model,loss_function, prediction_function, feature_cols,X_train,y_train,X_test,y_test,X_eval,y_eval,additional_params=None, train_pool=train_pool, test_pool=eval_pool):

    print("Model Type: Is it CatboostRanker?")
    print(isinstance(model, type(CatBoostRanker())))

    
    if isinstance(model, type(CatBoostRanker())) is True:   
        print("Model Type is CatboostRanker")
        parameters = deepcopy(default_parameters)
        parameters['loss_function'] = loss_function
        parameters['train_dir'] = loss_function
        print(parameters) 
        if additional_params is not None:
            parameters.update(additional_params)
        model = CatBoostRanker(**parameters)
        model.fit(train_pool, eval_set=test_pool, plot=True)
        print("best results (train on train):")
        print(model.get_best_score()["learn"])
        print("best results (on validation set):")
        print(model.get_best_score()["validation"])
    else:
        print("Model Type is CatboostClassifier")
        parameters = deepcopy(default_clf_parameters)
#        parameters['loss_function'] = loss_function
#        parameters['train_dir'] = loss_function
        model = CatBoostClassifier(**parameters)
        print(parameters) 
        model.fit(X_train, y_train,eval_set=(X_eval, y_eval),plot=True)        
        print("best results (train on train):")
        print(model.get_best_score()["learn"])
        print("best results (on validation set):")
        print(model.get_best_score()["validation"])
#Training predictions
    y_train_predictions = prediction_function(model, X_train)
    training_predictions = pd.DataFrame(X_train,columns=feature_cols)
    training_predictions['ActualRelevancyScore'] = y_train
    training_predictions['ActualRelevancy']=np.where(y_train==1.0,"Relevant","Not Relevant")

    if isinstance(model, type(CatBoostRanker())) is True:
        training_predictions['PredictedRelevancyScore'] = y_train_predictions
    else:
        training_predictions['Not_Relevant_prob'] = y_train_predictions[:,0]
        training_predictions['Yes_Relevant_prob'] = y_train_predictions[:,1]
        training_predictions['RelevanceLabelPrediction'] = np.argmax(y_train_predictions, axis=1).astype(np.float32)
        training_predictions['PredictedRelevancy']=np.where(training_predictions['RelevanceLabelPrediction']==1.0,"Relevant","Not Relevant")
        training_predictions['PredictedRelevancyScore'] = np.where(training_predictions['RelevanceLabelPrediction']==1.0,training_predictions['Yes_Relevant_prob'],training_predictions['Not_Relevant_prob'])
    
#Test predictions
    y_test_predictions = prediction_function(model, X_test)
    test_predictions = pd.DataFrame(X_test,columns=feature_cols)
    test_predictions['ActualRelevancyScore'] = y_test
    test_predictions['ActualRelevancy']=np.where(y_test==1.0,"Relevant","Not Relevant")

    if isinstance(model, type(CatBoostRanker())) is True:
        test_predictions['PredictedRelevancyScore'] = y_test_predictions
    else:
        test_predictions['Not_Relevant_prob'] = y_test_predictions[:,0]
        test_predictions['Yes_Relevant_prob'] = y_test_predictions[:,1]
        test_predictions['RelevanceLabelPrediction'] = np.argmax(y_test_predictions, axis=1).astype(np.float32)
        test_predictions['PredictedRelevancy']=np.where(test_predictions['RelevanceLabelPrediction']==1.0,"Relevant","Not Relevant")
        test_predictions['PredictedRelevancyScore'] = np.where(test_predictions['RelevanceLabelPrediction']==1.0,test_predictions['Yes_Relevant_prob'],test_predictions['Not_Relevant_prob'])
       

#     print("(Default) Feature importance (on train pool)")
#     display(model.get_feature_importance(data=train_pool,prettified=True).head(15))
    
#     try:
#         print("SHAP features importance, on all data:")
#         explainer = shap.TreeExplainer(model)
#         shap_values = explainer.shap_values(pd.concat([X_train,X_eval]),
#                                             y=pd.concat([y_train,y_eval]))

#         # # summarize the effects of all the features
#         shap.summary_plot(shap_values, pd.concat([X_train,X_eval]))
#    finally:
    return model, training_predictions , test_predictions  


In [314]:
def get_predicted_ranking_outcome(model, data):
#    a=model.predict(data)
#    print(a)
#    print(np.argmax(model.predict_proba(data), axis=1).astype(np.float32))
#    return np.argmax(model.predict_proba(data), axis=1).astype(np.float32)
    return model.predict(data)

In [315]:
def get_predicted_clf_outcome(model, data):
#    print(model.predict_proba(data))
#    print(np.argmax(model.predict_proba(data), axis=1).astype(np.float32))
#    return np.argmax(model.predict_proba(data), axis=1).astype(np.float32)
    return model.predict_proba(data)

In [310]:
#model = fit_model('RMSE', {'custom_metric': ['PrecisionAt:top=10', 'RecallAt:top=10', 'MAP:top=10']})
parameters = {}
ranking_model,ranking_training_predictions, ranking_test_predictions = fit_model(CatBoostRanker(),'QueryRMSE',get_predicted_ranking_outcome,feature_cols,X_train,y_train,X_test,y_test,X_eval,y_eval)


Model Type: Is it CatboostRanker?
True
Model Type is CatboostRanker
{'iterations': 2000, 'custom_metric': ['NDCG', 'AUC:type=Ranking', 'PrecisionAt:top=10', 'RecallAt:top=10', 'MAP:top=10'], 'verbose': False, 'random_seed': 42, 'metric_period': 4, 'save_snapshot': False, 'use_best_model': True, 'loss_function': 'QueryRMSE', 'train_dir': 'QueryRMSE'}


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

best results (train on train):
{'MAP:top=10': 0.37225463775116746, 'RecallAt:top=10': 0.9321943406300054, 'PrecisionAt:top=10': 0.06704770803142165, 'QueryRMSE': 0.1507495037944327}
best results (on validation set):
{'NDCG:type=Base': 0.6759827628642938, 'MAP:top=10': 0.27431559668401795, 'RecallAt:top=10': 0.8906882591093117, 'PrecisionAt:top=10': 0.06322697770066177, 'QueryRMSE': 0.16196556683244842, 'AUC:type=Ranking': 0.7718128243699502}


In [311]:
ranking_training_predictions.head()

,srch_id,site_id,visitor_location_country_id,visitor_hist_starrating,visitor_hist_adr_usd,prop_country_id,prop_id,prop_starrating,prop_review_score,prop_brand_bool,prop_location_score1,prop_location_score2,prop_log_historical_price,position,price_usd,promotion_flag,srch_destination_id,srch_length_of_stay,srch_booking_window,srch_adults_count,srch_children_count,srch_room_count,srch_saturday_night_bool,srch_query_affinity_score,orig_destination_distance,random_bool,comp1_rate,comp1_inv,comp1_rate_percent_diff,comp2_rate,comp2_inv,comp2_rate_percent_diff,comp3_rate,comp3_inv,comp3_rate_percent_diff,comp4_rate,comp4_inv,comp4_rate_percent_diff,comp5_rate,comp5_inv,comp5_rate_percent_diff,comp6_rate,comp6_inv,comp6_rate_percent_diff,comp7_rate,comp7_inv,comp7_rate_percent_diff,comp8_rate,comp8_inv,comp8_rate_percent_diff,predictionid,weekday,week_of_year,hour,minute,time_epoch,early_night,nans_count,ActualRelevancyScore,ActualRelevancy,PredictedRelevancyScore
0,71,24,216,NaN,NaN,98,76541,3,3.5,0,2.2,0.0482,3.21,1,16.9,0,341,1,18,1,1,1,1,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,11,23,14,1363130090.0,True,24,0.0,Not Relevant,0.033710
1,71,24,216,NaN,NaN,98,37933,3,4.0,0,2.08,0.0812,3.59,2,18.34,0,341,1,18,1,1,1,1,NaN,NaN,1,NaN,NaN,NaN,1.0,0.0,46.0,NaN,NaN,NaN,0.0,0.0,NaN,1.0,0.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,11,23,14,1363130090.0,True,20,0.0,Not Relevant,0.036360
2,71,24,216,NaN,NaN,98,108588,4,4.0,0,0.0,0.0268,4.08,3,26.78,1,341,1,18,1,1,1,1,NaN,NaN,1,NaN,NaN,NaN,1.0,0.0,108.0,1.0,0.0,32.0,-1.0,0.0,9.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,1,11,23,14,1363130090.0,True,17,0.0,Not Relevant,0.017548
3,71,24,216,NaN,NaN,98,28672,3,4.0,0,2.2,0.0151,3.35,4,21.48,0,341,1,18,1,1,1,1,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,1,11,23,14,1363130090.0,True,24,0.0,Not Relevant,0.010673
4,71,24,216,NaN,NaN,98,25741,3,3.5,0,1.61,0.0359,4.02,6,19.47,1,341,1,18,1,1,1,1,NaN,NaN,1,NaN,NaN,NaN,0.0,0.0,NaN,1.0,0.0,10.0,0.0,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,1,11,23,14,1363130090.0,True,19,0.0,Not Relevant,0.015136


In [316]:
parameters = {}
relevancyclf_model,relevancyclf_training_predictions, relevancyclf_test_predictions = fit_model(CatBoostClassifier(),'AUC',get_predicted_clf_outcome,feature_cols,X_train,y_train,X_test,y_test,X_eval,y_eval)


Model Type: Is it CatboostRanker?
False
Model Type is CatboostClassifier
{'iterations': 2000, 'random_seed': 42, 'learning_rate': 0.5, 'custom_loss': ['AUC', 'Accuracy']}


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 0.2074159	test: 0.2086029	best: 0.2086029 (0)	total: 21.6ms	remaining: 43.2s
1:	learn: 0.1266463	test: 0.1284016	best: 0.1284016 (1)	total: 41.9ms	remaining: 41.9s
2:	learn: 0.1131732	test: 0.1154236	best: 0.1154236 (2)	total: 57.6ms	remaining: 38.3s
3:	learn: 0.1097551	test: 0.1124045	best: 0.1124045 (3)	total: 72.2ms	remaining: 36s
4:	learn: 0.1081585	test: 0.1114645	best: 0.1114645 (4)	total: 85.1ms	remaining: 33.9s
5:	learn: 0.1075461	test: 0.1112727	best: 0.1112727 (5)	total: 94.5ms	remaining: 31.4s
6:	learn: 0.1071975	test: 0.1115514	best: 0.1112727 (5)	total: 105ms	remaining: 30s
7:	learn: 0.1070893	test: 0.1116050	best: 0.1112727 (5)	total: 117ms	remaining: 29.1s
8:	learn: 0.1067913	test: 0.1114940	best: 0.1112727 (5)	total: 129ms	remaining: 28.4s
9:	learn: 0.1066184	test: 0.1115702	best: 0.1112727 (5)	total: 144ms	remaining: 28.6s
10:	learn: 0.1065860	test: 0.1116119	best: 0.1112727 (5)	total: 175ms	remaining: 31.6s
11:	learn: 0.1062766	test: 0.1116563	best: 0.111272

100:	learn: 0.0900950	test: 0.1188535	best: 0.1111548 (29)	total: 1.28s	remaining: 24s
101:	learn: 0.0898852	test: 0.1189811	best: 0.1111548 (29)	total: 1.29s	remaining: 24s
102:	learn: 0.0897923	test: 0.1190640	best: 0.1111548 (29)	total: 1.3s	remaining: 23.9s
103:	learn: 0.0895697	test: 0.1191080	best: 0.1111548 (29)	total: 1.31s	remaining: 24s
104:	learn: 0.0893710	test: 0.1192001	best: 0.1111548 (29)	total: 1.33s	remaining: 24s
105:	learn: 0.0892158	test: 0.1192555	best: 0.1111548 (29)	total: 1.34s	remaining: 24s
106:	learn: 0.0891721	test: 0.1192768	best: 0.1111548 (29)	total: 1.35s	remaining: 24s
107:	learn: 0.0891051	test: 0.1193770	best: 0.1111548 (29)	total: 1.36s	remaining: 23.9s
108:	learn: 0.0890102	test: 0.1193372	best: 0.1111548 (29)	total: 1.38s	remaining: 23.9s
109:	learn: 0.0889358	test: 0.1193845	best: 0.1111548 (29)	total: 1.39s	remaining: 23.9s
110:	learn: 0.0887797	test: 0.1192519	best: 0.1111548 (29)	total: 1.4s	remaining: 23.9s
111:	learn: 0.0885945	test: 0.11926

197:	learn: 0.0763973	test: 0.1266629	best: 0.1111548 (29)	total: 2.59s	remaining: 23.6s
198:	learn: 0.0761924	test: 0.1264766	best: 0.1111548 (29)	total: 2.6s	remaining: 23.6s
199:	learn: 0.0761700	test: 0.1264600	best: 0.1111548 (29)	total: 2.61s	remaining: 23.5s
200:	learn: 0.0760609	test: 0.1261892	best: 0.1111548 (29)	total: 2.63s	remaining: 23.5s
201:	learn: 0.0759394	test: 0.1262015	best: 0.1111548 (29)	total: 2.65s	remaining: 23.5s
202:	learn: 0.0758949	test: 0.1261931	best: 0.1111548 (29)	total: 2.66s	remaining: 23.5s
203:	learn: 0.0757801	test: 0.1262527	best: 0.1111548 (29)	total: 2.67s	remaining: 23.5s
204:	learn: 0.0756094	test: 0.1262648	best: 0.1111548 (29)	total: 2.68s	remaining: 23.5s
205:	learn: 0.0754521	test: 0.1269125	best: 0.1111548 (29)	total: 2.69s	remaining: 23.4s
206:	learn: 0.0753826	test: 0.1268173	best: 0.1111548 (29)	total: 2.7s	remaining: 23.4s
207:	learn: 0.0751850	test: 0.1272420	best: 0.1111548 (29)	total: 2.72s	remaining: 23.4s
208:	learn: 0.0749630	t

...

1566:	learn: 0.0084673	test: 0.1886632	best: 0.1111548 (29)	total: 20.8s	remaining: 5.75s
1567:	learn: 0.0084650	test: 0.1887122	best: 0.1111548 (29)	total: 20.8s	remaining: 5.74s
1568:	learn: 0.0084650	test: 0.1887108	best: 0.1111548 (29)	total: 20.8s	remaining: 5.72s
1569:	learn: 0.0084546	test: 0.1886170	best: 0.1111548 (29)	total: 20.8s	remaining: 5.71s
1570:	learn: 0.0084472	test: 0.1887562	best: 0.1111548 (29)	total: 20.9s	remaining: 5.7s
1571:	learn: 0.0084229	test: 0.1887223	best: 0.1111548 (29)	total: 20.9s	remaining: 5.68s
1572:	learn: 0.0084080	test: 0.1886912	best: 0.1111548 (29)	total: 20.9s	remaining: 5.67s
1573:	learn: 0.0084002	test: 0.1887118	best: 0.1111548 (29)	total: 20.9s	remaining: 5.66s
1574:	learn: 0.0083887	test: 0.1886398	best: 0.1111548 (29)	total: 20.9s	remaining: 5.65s
1575:	learn: 0.0083783	test: 0.1890862	best: 0.1111548 (29)	total: 21s	remaining: 5.64s
1576:	learn: 0.0083599	test: 0.1893378	best: 0.1111548 (29)	total: 21s	remaining: 5.62s
1577:	learn: 0.

1661:	learn: 0.0075146	test: 0.1913149	best: 0.1111548 (29)	total: 22.2s	remaining: 4.51s
1662:	learn: 0.0075135	test: 0.1913259	best: 0.1111548 (29)	total: 22.2s	remaining: 4.5s
1663:	learn: 0.0075128	test: 0.1913367	best: 0.1111548 (29)	total: 22.2s	remaining: 4.48s
1664:	learn: 0.0075124	test: 0.1913446	best: 0.1111548 (29)	total: 22.2s	remaining: 4.47s
1665:	learn: 0.0075100	test: 0.1913870	best: 0.1111548 (29)	total: 22.2s	remaining: 4.46s
1666:	learn: 0.0075085	test: 0.1914123	best: 0.1111548 (29)	total: 22.2s	remaining: 4.44s
1667:	learn: 0.0074896	test: 0.1914608	best: 0.1111548 (29)	total: 22.3s	remaining: 4.43s
1668:	learn: 0.0074891	test: 0.1914803	best: 0.1111548 (29)	total: 22.3s	remaining: 4.42s
1669:	learn: 0.0074782	test: 0.1915461	best: 0.1111548 (29)	total: 22.3s	remaining: 4.4s
1670:	learn: 0.0074714	test: 0.1915416	best: 0.1111548 (29)	total: 22.3s	remaining: 4.39s
1671:	learn: 0.0074613	test: 0.1915942	best: 0.1111548 (29)	total: 22.3s	remaining: 4.38s
1672:	learn:

1758:	learn: 0.0067133	test: 0.1949322	best: 0.1111548 (29)	total: 23.5s	remaining: 3.21s
1759:	learn: 0.0066971	test: 0.1949691	best: 0.1111548 (29)	total: 23.5s	remaining: 3.2s
1760:	learn: 0.0066971	test: 0.1949699	best: 0.1111548 (29)	total: 23.5s	remaining: 3.19s
1761:	learn: 0.0066859	test: 0.1951971	best: 0.1111548 (29)	total: 23.5s	remaining: 3.17s
1762:	learn: 0.0066859	test: 0.1951962	best: 0.1111548 (29)	total: 23.5s	remaining: 3.16s
1763:	learn: 0.0066816	test: 0.1952402	best: 0.1111548 (29)	total: 23.5s	remaining: 3.15s
1764:	learn: 0.0066800	test: 0.1952144	best: 0.1111548 (29)	total: 23.5s	remaining: 3.13s
1765:	learn: 0.0066653	test: 0.1952916	best: 0.1111548 (29)	total: 23.5s	remaining: 3.12s
1766:	learn: 0.0066475	test: 0.1953851	best: 0.1111548 (29)	total: 23.6s	remaining: 3.11s
1767:	learn: 0.0066405	test: 0.1953842	best: 0.1111548 (29)	total: 23.6s	remaining: 3.1s
1768:	learn: 0.0066315	test: 0.1953421	best: 0.1111548 (29)	total: 23.6s	remaining: 3.08s
1769:	learn:

1855:	learn: 0.0061417	test: 0.1989281	best: 0.1111548 (29)	total: 24.7s	remaining: 1.91s
1856:	learn: 0.0061373	test: 0.1989289	best: 0.1111548 (29)	total: 24.7s	remaining: 1.9s
1857:	learn: 0.0061243	test: 0.1990998	best: 0.1111548 (29)	total: 24.7s	remaining: 1.89s
1858:	learn: 0.0061142	test: 0.1992744	best: 0.1111548 (29)	total: 24.7s	remaining: 1.87s
1859:	learn: 0.0060994	test: 0.1993282	best: 0.1111548 (29)	total: 24.7s	remaining: 1.86s
1860:	learn: 0.0060911	test: 0.1994793	best: 0.1111548 (29)	total: 24.7s	remaining: 1.85s
1861:	learn: 0.0060885	test: 0.1995329	best: 0.1111548 (29)	total: 24.7s	remaining: 1.83s
1862:	learn: 0.0060831	test: 0.1995738	best: 0.1111548 (29)	total: 24.8s	remaining: 1.82s
1863:	learn: 0.0060784	test: 0.1996236	best: 0.1111548 (29)	total: 24.8s	remaining: 1.81s
1864:	learn: 0.0060783	test: 0.1996275	best: 0.1111548 (29)	total: 24.8s	remaining: 1.79s
1865:	learn: 0.0060773	test: 0.1996162	best: 0.1111548 (29)	total: 24.8s	remaining: 1.78s
1866:	learn

1949:	learn: 0.0055738	test: 0.2006378	best: 0.1111548 (29)	total: 25.9s	remaining: 663ms
1950:	learn: 0.0055683	test: 0.2006042	best: 0.1111548 (29)	total: 25.9s	remaining: 650ms
1951:	learn: 0.0055656	test: 0.2005761	best: 0.1111548 (29)	total: 25.9s	remaining: 637ms
1952:	learn: 0.0055607	test: 0.2005894	best: 0.1111548 (29)	total: 25.9s	remaining: 623ms
1953:	learn: 0.0055596	test: 0.2005723	best: 0.1111548 (29)	total: 25.9s	remaining: 610ms
1954:	learn: 0.0055537	test: 0.2006095	best: 0.1111548 (29)	total: 25.9s	remaining: 597ms
1955:	learn: 0.0055470	test: 0.2007615	best: 0.1111548 (29)	total: 25.9s	remaining: 584ms
1956:	learn: 0.0055371	test: 0.2008954	best: 0.1111548 (29)	total: 26s	remaining: 571ms
1957:	learn: 0.0055335	test: 0.2009450	best: 0.1111548 (29)	total: 26s	remaining: 557ms
1958:	learn: 0.0055330	test: 0.2009565	best: 0.1111548 (29)	total: 26s	remaining: 544ms
1959:	learn: 0.0055275	test: 0.2009263	best: 0.1111548 (29)	total: 26s	remaining: 531ms
1960:	learn: 0.005

In [317]:
ranking_training_predictions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92451 entries, 0 to 92450
Data columns (total 61 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   srch_id                      92451 non-null  object 
 1   site_id                      92451 non-null  object 
 2   visitor_location_country_id  92451 non-null  object 
 3   visitor_hist_starrating      4461 non-null   object 
 4   visitor_hist_adr_usd         4486 non-null   object 
 5   prop_country_id              92451 non-null  object 
 6   prop_id                      92451 non-null  object 
 7   prop_starrating              92451 non-null  object 
 8   prop_review_score            92300 non-null  object 
 9   prop_brand_bool              92451 non-null  object 
 10  prop_location_score1         92451 non-null  object 
 11  prop_location_score2         72046 non-null  object 
 12  prop_log_historical_price    92451 non-null  object 
 13  position        

In [318]:
relevancyclf_training_predictions.head()

,srch_id,site_id,visitor_location_country_id,visitor_hist_starrating,visitor_hist_adr_usd,prop_country_id,prop_id,prop_starrating,prop_review_score,prop_brand_bool,prop_location_score1,prop_location_score2,prop_log_historical_price,position,price_usd,promotion_flag,srch_destination_id,srch_length_of_stay,srch_booking_window,srch_adults_count,srch_children_count,srch_room_count,srch_saturday_night_bool,srch_query_affinity_score,orig_destination_distance,random_bool,comp1_rate,comp1_inv,comp1_rate_percent_diff,comp2_rate,comp2_inv,comp2_rate_percent_diff,comp3_rate,comp3_inv,comp3_rate_percent_diff,comp4_rate,comp4_inv,comp4_rate_percent_diff,comp5_rate,comp5_inv,comp5_rate_percent_diff,comp6_rate,comp6_inv,comp6_rate_percent_diff,comp7_rate,comp7_inv,comp7_rate_percent_diff,comp8_rate,comp8_inv,comp8_rate_percent_diff,predictionid,weekday,week_of_year,hour,minute,time_epoch,early_night,nans_count,ActualRelevancyScore,ActualRelevancy,Not_Relevant_prob,Yes_Relevant_prob,RelevanceLabelPrediction,PredictedRelevancy,PredictedRelevancyScore
0,71,24,216,NaN,NaN,98,76541,3,3.5,0,2.2,0.0482,3.21,1,16.9,0,341,1,18,1,1,1,1,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,11,23,14,1363130090.0,True,24,0.0,Not Relevant,0.978216,0.021784,0.0,Not Relevant,0.978216
1,71,24,216,NaN,NaN,98,37933,3,4.0,0,2.08,0.0812,3.59,2,18.34,0,341,1,18,1,1,1,1,NaN,NaN,1,NaN,NaN,NaN,1.0,0.0,46.0,NaN,NaN,NaN,0.0,0.0,NaN,1.0,0.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,11,23,14,1363130090.0,True,20,0.0,Not Relevant,0.973249,0.026751,0.0,Not Relevant,0.973249
2,71,24,216,NaN,NaN,98,108588,4,4.0,0,0.0,0.0268,4.08,3,26.78,1,341,1,18,1,1,1,1,NaN,NaN,1,NaN,NaN,NaN,1.0,0.0,108.0,1.0,0.0,32.0,-1.0,0.0,9.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,1,11,23,14,1363130090.0,True,17,0.0,Not Relevant,0.996076,0.003924,0.0,Not Relevant,0.996076
3,71,24,216,NaN,NaN,98,28672,3,4.0,0,2.2,0.0151,3.35,4,21.48,0,341,1,18,1,1,1,1,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,1,11,23,14,1363130090.0,True,24,0.0,Not Relevant,0.992027,0.007973,0.0,Not Relevant,0.992027
4,71,24,216,NaN,NaN,98,25741,3,3.5,0,1.61,0.0359,4.02,6,19.47,1,341,1,18,1,1,1,1,NaN,NaN,1,NaN,NaN,NaN,0.0,0.0,NaN,1.0,0.0,10.0,0.0,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,1,11,23,14,1363130090.0,True,19,0.0,Not Relevant,0.976626,0.023374,0.0,Not Relevant,0.976626


In [319]:
ranking_training_predictions["Predictedrank"] = ranking_training_predictions.groupby("srch_id")["PredictedRelevancyScore"].rank("dense", ascending=False)
ranking_training_predictions_df=ranking_training_predictions.sort_values(by=['srch_id','position'], ascending=True)
ranking_training_predictions_df = ranking_training_predictions_df.reset_index(drop=True)

ranking_training_predictions_df[['Not_Relevant_prob','Yes_Relevant_prob','RelevanceLabelPrediction','PredictedRelevancy','PredictedScore']] = relevancyclf_training_predictions[['Not_Relevant_prob','Yes_Relevant_prob','RelevanceLabelPrediction','PredictedRelevancy','PredictedRelevancyScore']]

ranking_training_predictions_df['PredictedRelevancy'].value_counts()

Not Relevant    92403
Relevant           48
Name: PredictedRelevancy, dtype: int64

In [320]:
ranking_test_predictions["Predictedrank"] = ranking_test_predictions.groupby("srch_id")["PredictedRelevancyScore"].rank("dense", ascending=False)
ranking_test_predictions_df=ranking_test_predictions.sort_values(by=['srch_id','position'], ascending=True)
ranking_test_predictions_df = ranking_test_predictions_df.reset_index(drop=True)

ranking_test_predictions_df[['Not_Relevant_prob','Yes_Relevant_prob','RelevanceLabelPrediction','PredictedRelevancy','PredictedScore']] = relevancyclf_test_predictions[['Not_Relevant_prob','Yes_Relevant_prob','RelevanceLabelPrediction','PredictedRelevancy','PredictedRelevancyScore']]

ranking_test_predictions_df.head()


,srch_id,site_id,visitor_location_country_id,visitor_hist_starrating,visitor_hist_adr_usd,prop_country_id,prop_id,prop_starrating,prop_review_score,prop_brand_bool,prop_location_score1,prop_location_score2,prop_log_historical_price,position,price_usd,promotion_flag,srch_destination_id,srch_length_of_stay,srch_booking_window,srch_adults_count,srch_children_count,srch_room_count,srch_saturday_night_bool,srch_query_affinity_score,orig_destination_distance,random_bool,comp1_rate,comp1_inv,comp1_rate_percent_diff,comp2_rate,comp2_inv,comp2_rate_percent_diff,comp3_rate,comp3_inv,comp3_rate_percent_diff,comp4_rate,comp4_inv,comp4_rate_percent_diff,comp5_rate,comp5_inv,comp5_rate_percent_diff,comp6_rate,comp6_inv,comp6_rate_percent_diff,comp7_rate,comp7_inv,comp7_rate_percent_diff,comp8_rate,comp8_inv,comp8_rate_percent_diff,predictionid,weekday,week_of_year,hour,minute,time_epoch,early_night,nans_count,ActualRelevancyScore,ActualRelevancy,PredictedRelevancyScore,Predictedrank,Not_Relevant_prob,Yes_Relevant_prob,RelevanceLabelPrediction,PredictedRelevancy,PredictedScore
0,246,14,100,NaN,NaN,219,11396,4,5.0,0,0.69,0.0551,5.36,1,159.64,0,18294,1,6,2,0,1,0,NaN,87.17,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,10,9,55,1362390943.0,False,27,1.0,Relevant,0.132251,1.0,0.853474,0.146526,0.0,Not Relevant,0.853474
1,246,14,100,NaN,NaN,219,103885,2,4.0,1,1.1,0.154,4.75,2,98.32,0,18294,1,6,2,0,1,0,NaN,93.87,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,10,9,55,1362390943.0,False,25,0.0,Not Relevant,0.117169,2.0,0.834611,0.165389,0.0,Not Relevant,0.834611
2,246,14,100,NaN,NaN,219,61167,3,4.5,0,1.1,0.0314,4.87,3,101.24,0,18294,1,6,2,0,1,0,NaN,92.31,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,0,10,9,55,1362390943.0,False,25,0.0,Not Relevant,0.052576,3.0,0.914844,0.085156,0.0,Not Relevant,0.914844
3,246,14,100,NaN,NaN,219,95490,2,3.5,1,1.1,0.0142,4.88,4,95.4,0,18294,1,6,2,0,1,0,NaN,89.28,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,0,10,9,55,1362390943.0,False,27,0.0,Not Relevant,0.010497,6.0,0.940705,0.059295,0.0,Not Relevant,0.940705
4,246,14,100,NaN,NaN,219,127213,3,4.0,1,1.1,0.1652,4.95,6,119.73,0,18294,1,6,2,0,1,0,NaN,93.97,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,0,10,9,55,1362390943.0,False,25,0.0,Not Relevant,0.039523,4.0,0.872000,0.128000,0.0,Not Relevant,0.872000


In [321]:
ranking_training_predictions_df.to_csv('./data/hotelsearch_training_predictions.csv')
ranking_test_predictions_df.to_csv('./data/hotelsearch_test_predictions.csv')

In [278]:
#relevancyclf_training_predictions.to_csv('./data/hotelsearch_clf_training.csv')
#relevancyclf_test_predictions.to_csv('./data/hotelsearch_clf_test.csv')

In [279]:
ranking_training_predictions_df.shape

(92451, 64)